In [18]:
import pandas as pd
import numpy as np

In [19]:
averaged_car_measurements = pd.read_csv('data/averaged_car_measurements.csv')

# Change date to datetime, format is %Y-%m-%d
averaged_car_measurements['date'] = pd.to_datetime(averaged_car_measurements['date'], format='%Y-%m-%d')

averaged_car_measurements.head()

,Hour,date,NO_ugm3,NO2_ugm3,O3_ugm3,CO_mgm3,CO2_mgm3,PM25_ugm3,SiteID,Lat,Long,month,day_of_week
0,8,2021-06-28,12.044255,0.085176,39.303789,0.351863,878.897362,5.072396,1,53.348754,-6.257607,6,0
1,9,2021-06-28,5.263025,9.600681,29.413390,0.350409,842.139168,7.998587,1,53.348754,-6.257607,6,0
2,9,2021-08-16,10.318074,3.006013,61.380364,0.380456,809.073556,6.122852,1,53.348754,-6.257607,8,0
3,9,2021-10-22,456.213318,NaN,NaN,0.432339,850.479574,5.752877,1,53.348754,-6.257607,10,4
4,9,2022-07-29,15.093107,NaN,NaN,0.415959,805.524520,4.757151,1,53.348754,-6.257607,7,4


In [20]:
# Prep hourly_weather data
hourly_weather = pd.read_csv('Weather-Air_pollution/merged_weather_data.csv')

# Convert hour to integer
hourly_weather['hour'] = hourly_weather['hour'].str.replace(':00', '')
hourly_weather['hour'] = hourly_weather['hour'].astype(int)

# Drop latitute, longitude and datetime columns as they are not needed
hourly_weather = hourly_weather.drop(['latitude', 'longitude','datetime'], axis=1)

# Change date to datetime, format is %d/%m/%Y
hourly_weather['date'] = pd.to_datetime(hourly_weather['date'], format='%d/%m/%Y')

hourly_weather.head()

,date,hour,avgtempC,maxtempC,mintempC,sunHour,uvIndex,humidity,winddirDegree,windspeedKmph,cloudcover,precipMM,pressure
0,2021-05-01,0,6,8,3,13,3,80,305,9,5,0.0,1017
1,2021-05-01,1,6,8,3,13,3,82,306,10,15,0.0,1017
2,2021-05-01,2,6,8,3,13,3,83,306,11,25,0.0,1017
3,2021-05-01,3,6,8,3,13,3,85,307,12,35,0.0,1017
4,2021-05-01,4,6,8,3,13,3,84,308,12,41,0.0,1017


Traffic Data

In [21]:
#Prepare hourly traffic data
hourly_traffic = pd.read_csv('Traffic/hourly_traffic_volume.csv')

#Change date to datetime, format is %Y-%m-%d
hourly_traffic['Date'] = pd.to_datetime(hourly_traffic['Date'], format='%Y-%m-%d')

hourly_traffic.head()

,Site,Date,Hour,Sum_Volume
0,1,2022-04-01,0,169
1,1,2022-04-01,1,134
2,1,2022-04-01,2,56
3,1,2022-04-01,3,31
4,1,2022-04-01,4,40


Weather data

In [30]:
# Left join the two dataframes on hour and date together
Full_data = pd.merge(averaged_car_measurements, hourly_weather, how='left', left_on=['date','Hour'], right_on=['date', 'hour'])
Full_data.head()

# Left join the air pollution sensors data
air_pollution_sensors = pd.read_csv('dublin_official_measurements/dublin_official_measurements_sonitus_api.csv')
# Extract hour and date from datetime column
air_pollution_sensors['hour'] = pd.to_datetime(air_pollution_sensors['datetime']).dt.hour
air_pollution_sensors['date'] = pd.to_datetime(air_pollution_sensors['datetime']).dt.date
# Drop datetime column
air_pollution_sensors = air_pollution_sensors.drop(['datetime'], axis=1)
#convert the date column to datetime
air_pollution_sensors['date'] = pd.to_datetime(air_pollution_sensors['date'])
Full_data = pd.merge(Full_data, air_pollution_sensors, how='left', left_on=['date','Hour'], right_on=['date', 'hour'])

##Traffic data
#Left join the two dataframes on hour and date together
Full_data = pd.merge(Full_data, hourly_traffic, how='left', left_on=['date','Hour','SiteID'], right_on=['Date', 'Hour','Site'])
#Drop hour and date and site columns
Full_data = Full_data.drop(['Site','hour_y','hour_x','Date'], axis=1)

#Drop columns with more than 50% of missing values
Full_data = Full_data.dropna(thresh=0.5*len(Full_data), axis=1)

In [31]:
Full_data.columns

Index(['Hour', 'date', 'NO_ugm3', 'NO2_ugm3', 'O3_ugm3', 'CO_mgm3', 'CO2_mgm3',
       'PM25_ugm3', 'SiteID', 'Lat', 'Long', 'month', 'day_of_week',
       'avgtempC', 'maxtempC', 'mintempC', 'sunHour', 'uvIndex', 'humidity',
       'winddirDegree', 'windspeedKmph', 'cloudcover', 'precipMM', 'pressure',
       'DCC-AQ1-co', 'DCC-AQ1-no', 'DCC-AQ10-no', 'DCC-AQ13-no', 'DCC-AQ5-no',
       'DCC-AQ6-no', 'DCC-AQ1-no2', 'DCC-AQ10-no2', 'DCC-AQ13-no2',
       'DCC-AQ22-no2', 'DCC-AQ5-no2', 'DCC-AQ6-no2', 'DCC-AQ69-no2',
       'DCC-AQ22-o3', 'DCC-AQ69-o3', 'DCC-AQ10-pm1', 'DCC-AQ13-pm1',
       'DCC-AQ2-pm1', 'DCC-AQ3-pm1', 'DCC-AQ4-pm1', 'DCC-AQ5-pm1',
       'DCC-AQ52-pm1', 'DCC-AQ6-pm1', 'TNO2161-pm1', 'TNO2162-pm1',
       'TNO4435-pm1', 'TNT1088-pm1', 'DCC-AQ10-pm10', 'DCC-AQ13-pm10',
       'DCC-AQ2-pm10', 'DCC-AQ22-pm10', 'DCC-AQ3-pm10', 'DCC-AQ4-pm10',
       'DCC-AQ5-pm10', 'DCC-AQ52-pm10', 'DCC-AQ6-pm10', 'TNO2161-pm10',
       'TNO2162-pm10', 'TNO4435-pm10', 'TNT1088-pm10', 'DCC-

In [32]:
#Change full data to be sorted by date
#import full data again
Full_data = Full_data.sort_values(by='date')

#drop rows where PM25_ugm3 is na
Full_data = Full_data.dropna(subset=['PM25_ugm3'])

#Drop month column
Full_data = Full_data.drop(['month'], axis=1)

# Export to csv
Full_data.to_csv('prepped_data/Full_data.csv', index=False)